<a href="https://colab.research.google.com/github/ruchirlives/Python/blob/main/MySim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating a project portfolio
Lets get the simPy simulation package

In [ ]:
# @title Initialisations
!pip install simpy

import simpy, yaml
import pandas as pd
import json
from collections import defaultdict
from uuid import uuid4
# Set the option to adjust display width to not limit the total width of the DataFrame display

# Set the maximum number of rows and columns to display
pd.set_option('display.max_rows', 100)  # Default is 60
pd.set_option('display.max_columns', 19)  # Default is 20

# Set the width and the maximum columns width
pd.set_option('display.width', 1000)  # Adjust accordingly
pd.set_option('display.max_colwidth', 100)  # Adjust accordingly
# Set display precision to 2 decimal places
pd.set_option('display.precision', 2)


from google.colab import data_table
data_table.enable_dataframe_formatter()
data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:.2f}"

Lets get the **helper** functions we need.

In [ ]:
# @title Helper functions
ALL_MONTHS = [
 "jan",
 "feb",
 "mar",
 "apr",
 "may",
 "jun",
 "jul",
 "aug",
 "sep",
 "oct",
 "nov",
 "dec",
]


def get_current_month(start_month='apr', month=0):
	# Start month for the financial year is always April, which is the 4th month, index 3 in ALL_MONTHS
	# Since env.now starts from 1 for April, subtract 1 to align with zero-based indexing
	elapsed_months_adjusted = month

	# Calculate the index of the current month
	# April's index (3) is used as a fixed starting point for financial year starting in April
	current_month_index = (3 + elapsed_months_adjusted) % len(ALL_MONTHS)

	# Return the name of the current month
	return ALL_MONTHS[current_month_index]


def printtimestamp(env):
	start_month = 'apr'
	month = get_current_month(start_month, env.now - 1)
	print(f"\nMonth: {env.now} ({month})")

def pivotbudget(db):
    # Pivot the DataFrame
    df = db.pivot_table(index=['item'], columns=['step'], values='budget', aggfunc='sum', fill_value=0)

    # Create lookup dictionaries for 'description' and 'type'
    lookup_dict_description = {row['item']: row['description'] if pd.notnull(row['description']) else '' for index, row in db.iterrows()}
    lookup_dict_type = {row['item']: row['type'] if pd.notnull(row['type']) else '' for index, row in db.iterrows()}

    # Map 'description' and 'type' to the DataFrame
    df['description'] = df.index.map(lookup_dict_description)
    df['type'] = df.index.map(lookup_dict_type)

    # Replace NaN values with a blank string
    df['description'] = df['description'].fillna('')
    df['type'] = df['type'].fillna('')

    # Get a list of your current DataFrame columns excluding 'description' and 'type'
    columns_except_extra = [col for col in df.columns if col not in ['description', 'type']]

    # Define the new order, placing 'description' and 'type' first
    new_column_order = ['description', 'type'] + columns_except_extra

    # Reorder the DataFrame columns
    df = df[new_column_order]

    # Reverse the DataFrame
    pf = df.iloc[::-1]

    # Sort by type column ascending
    pf = pf.sort_values(by='type', ascending=True)

    return pf

def parseYAML(yamltext):

    # |SubFunction to map 'cls' strings to function objects
    def map_cls_strings_to_objects(data):
      if isinstance(data, list):
          for index, item in enumerate(data):
              data[index] = map_cls_strings_to_objects(
                  item)  # Assign the returned value in case of lists
      elif isinstance(data, dict):
          for key, value in data.items():
              if key == 'cls' and isinstance(value, str):
                  # Attempt to replace string with class from globals
                  data[key] = globals().get(value, value)  # Fallback
              else:
                  data[key] = map_cls_strings_to_objects(
                      value)  # Recurse into nested structures
      return data  # Important: return the modified data

    data = yaml.safe_load(yamltext)
    map_cls_strings_to_objects(data)
    return data

def yaml_to_react_flow_json(yaml_file_path, json_file_path=None):
    # Load the YAML data
    with open(yaml_file_path, "r") as file:
        yaml_data = yaml.safe_load(file)

    # Your existing conversion logic here
    def yaml_to_react_flow(yaml_data):
        nodes = []
        edges = []  # Now we'll use edges to link list items to their parent node

        for index, phase in enumerate(yaml_data):
            # Generate a unique ID for each parent node
            parent_node_id = str(uuid4())
            attributes = []

            # Iterate through each item in the phase
            for key, value in phase.items():
                if not isinstance(value, list):
                    # Convert non-list items to a string and add to the attributes list
                    attributes.append(f"{key}: {value}")
                else:
                    # Handle list items by creating child nodes and linking them to the parent
                    for item in value:  # each new node
                        subattribute = []
                        for subkey in item:
                            subattribute.append(f'{subkey}: {item[subkey]}')

                        child_node_id = str(uuid4())
                        child_node = {
                            "id": child_node_id,
                            "type": "UMLClassNode",  # Adjust as needed for your visualization
                            "position": {
                                "x": 250 * index + 100,
                                "y": 200,
                            },  # Adjust positioning as needed
                            "data": {
                                "name": f"{key}",
                                "attributes": (
                                    subattribute
                                ),
                            },
                        }
                        nodes.append(child_node)
                        # Create an edge from the parent node to the child node
                        edges.append(
                            {
                                "id": str(uuid4()),
                                "source": parent_node_id,
                                "target": child_node_id,
                            }
                        )

            # Create the parent node
            node = {
                "id": parent_node_id,
                "type": "UMLClassNode",
                "position": {"x": 250 * index, "y": 100},
                "data": {
                    "name": phase["name"],
                    "attributes": attributes,
                },
            }
            nodes.append(node)

        return {"nodes": nodes, "edges": edges}

    # Convert the YAML content to React Flow JSON format
    react_flow_data = yaml_to_react_flow(yaml_data)

    # Serialize the data to a JSON file - optionally
    if json_file_path:
      with open(json_file_path, "w") as file:
          json.dump(react_flow_data, file, indent=4)

    return(react_flow_data)

#yaml_to_react_flow_json('data.yaml', 'data.json')



In [ ]:
# @title Import Export
def react_flow_to_yaml(json_file_path, yaml_file_path=None):
    # Load the React Flow JSON data
    with open(json_file_path, 'r') as json_file:
        react_flow_data = json.load(json_file)

    nodes = react_flow_data['nodes']
    edges = react_flow_data['edges']

    # Create mappings from node ID to node data for easy lookup
    node_data_map = {node['id']: node for node in nodes}

    # Initialize storage for YAML data
    yaml_data = []

    # Find parent nodes (nodes without incoming edges)
    parent_ids = set(node_data_map.keys()) - set(edge['target'] for edge in edges)

    for parent_id in parent_ids:
        parent_node = node_data_map[parent_id]
        phase_data = {}

        # Extract attributes for the parent node
        for attr in parent_node['data']['attributes']:
            print(attr)
            key, value = attr.split(': ', 1)
            # Convert numeric values back if applicable
            try:
                if value.isdigit():
                    value = int(value)
                else:
                    # Attempt to evaluate any expressions or convert to float
                    value = eval(value)
            except:
                pass
            phase_data[key] = value

        # Find and organize child nodes under their respective categories
        child_edges = [edge for edge in edges if edge['source'] == parent_id]
        for edge in child_edges:
            child_node = node_data_map[edge['target']]
            category = child_node['data']['name']
            child_attrs = {}

            for attr in child_node['data']['attributes']:
                # Check if attr contains ': ', indicating a key-value pair
                if ': ' in attr:
                    key, value = attr.split(': ', 1)
                    try:
                        # Attempt to convert string numbers to int or evaluate expressions
                        if value.isdigit():
                            value = int(value)
                        elif value.startswith('{') and value.endswith('}'):
                            value = value
                        else:
                            value = str(value)
                    except:
                        # If eval fails, keep value as is (likely a string)
                        pass
                else:
                    # Handle the case where attr does not contain ': '
                    # Option 1: Skip this attribute
                    # continue

                    # Option 2: Use a default key, or handle it differently
                    key = 'unknown'  # Or any other default key or handling logic
                    value = attr  # Store the entire attr as value

                # Store the key-value pair in child_attrs dictionary
                child_attrs[key] = value

            # Check if the category exists in phase_data, if not, add it
            if category not in phase_data:
                phase_data[category] = []
            # Append the child_attrs dictionary to the appropriate category in phase_data
            phase_data[category].append(child_attrs)


        yaml_data.append(phase_data)

    # Serialize the reconstructed data to YAML
    yaml_output = yaml.dump(yaml_data, allow_unicode=True, sort_keys=False, default_flow_style=False)

    # Optionally, write the YAML data to a file
    if yaml_file_path:
        with open(yaml_file_path, 'w', encoding='utf-8') as yaml_file:
            yaml_file.write(yaml_output)

    return yaml_output

# react_flow_to_yaml('data.json', 'data.yaml')

Get the staff classes and associated Global variables

In [ ]:
# @title Staff Classes
#from functions import *

NIRATE = 0.138
NITHRESHOLD = 175
EMPLOYERPENSIONRATE = 0.09
PENSIONFTETHRESHOLD = 0.2
REALLIVINGWAGE = 12
FCRDATA = []
SUPPORTDATA = []

#Need to change all the below to **kwargs

class worker:
  def __init__(self,   **kwargs):
    self.position = kwargs.get('position','undesignated')
    self.name = kwargs.get('name','staff member')
    self.age = kwargs.get('age',49)
    self.department = kwargs.get('department','unspecified')
    self.mobilephone = kwargs.get('mobilephone','not assigned')
    self.linemanagerrate = kwargs.get('linemanagerrate',0)

    self.fte_salary = kwargs.get('salary',0)
    self.fte = kwargs.get('fte',1)
    self.salary = self.fte * self.fte_salary

  def info(self):
    for attr, value in self.__dict__.items():
      print(f"{attr} : {value}")

  def getbreakdown(self, month):
    data=[]
    salary = self.getMonthSalary(month)
    data.append({'step':month,'item': 'salary', 'budget':salary,'type':'1. Staffing'})
    data.append({'step':month,'item': 'ni', 'budget':self.getNI(salary), 'type':'1. Staffing'})
    data.append({'step':month,'item': 'pension', 'budget':self.getPension(salary, self.fte), 'type':'1. Staffing'})

    return(data)

  def getSalaryCost(self):
    salary = self.salary
    monthlysalary = salary / 12
    monthlycost = (
        monthlysalary
          + self.getNI(monthlysalary)
          + self.getPension(monthlysalary, self.fte)
    )
    annualcost = monthlycost * 12
    annualcost = (annualcost)
    return annualcost

  def getMonthSalaryCost(self, month):
    return (self.getSalaryCost() / 12)

  def getMonthSalary(self, month):
    return (self.salary / 12)

  def getNI(self, monthlySalary):
    niRate = NIRATE
    threshold = NITHRESHOLD
    monthlyThreshold = threshold / 7 * 365 / 12

    if self.salary > monthlyThreshold:
      ni = max(0, (monthlySalary - monthlyThreshold)) * niRate
    else:
      ni = 0
    return (ni)

  def getPension(self, salary, fte):
    pensionRate = EMPLOYERPENSIONRATE
    if fte > PENSIONFTETHRESHOLD:
      pension = salary * pensionRate
    else:
      pension = 0

    return pension

Lets get the **project** classes that specify which projects we can instantiate. The root class is called **project** and its subclasses are combined in different ways to create the other projects.

In [ ]:
# @title Project Classes
#from functions import *
#from staff import worker


class project:

	def __init__(self, portfolio, env, **kwargs):
		self.kwargs = kwargs
		self.name = kwargs.get('name','New Project')
		self.term = kwargs.get('term',0)
		self.directcosts = kwargs.get('directcosts',[])
		self.supports = kwargs.get('supports',[])
		self.env = env
		self.portfolio = portfolio
		self.startstep = env.now
		self.consolidated_account = portfolio.consolidated_account
		self.budget = kwargs.get('budget', 0)

		self.policies=[]
		policies = kwargs.get('policies')

		if policies:
			for policy in policies:
				cls=globals().get(policy['policy'])
				if callable(cls):
					self.policies.append(cls(self.env, self, **policy))

		self.staff = []
		staffing = kwargs.get('staffing',[])
		for person in staffing:
			self.addstaff(worker(**person))

		self.costs_thismonth = 0
		self.income_thismonth = 0
		self.cost = 0
		self.income = 0
		self.env.process(self.start())

	def calculate(self, step):
		dcosts = self.getdirectcosts(step)
		directcost = sum([d['budget'] for d in dcosts if 'budget' in d])

		scosts = self.getsupports(step)
		supportcost = sum([d['budget'] for d in scosts if 'budget' in d])

		self.costs_thismonth += self.getsalarycosts(step) + directcost + supportcost  #self.budget / self.term
		self.income_thismonth += 0 #self.costs_thismonth * 1.5 #Very simply estimation of income_thismonth generated for the base class

		return

	def getsupports(self, step):
		supports = self.supports
		costs = []

		for support in supports:
			item = support.get('item','unspecified')
			applystep = support.get('step',0)
			description = support.get('description', '')
			freq = support.get('frequency','oneoff')

			matching_dictionaries = [d for d in SUPPORTDATA if d.get('item') == item]
			eligiblestep = freq=='monthly' or (freq=='oneoff' and applystep==step) or (freq=='annual' and (step-applystep) % 12 == 0)

			if eligiblestep and len(matching_dictionaries)>0:
				lookup = matching_dictionaries[0]
				cost = support['units'] * lookup['dayrate'] * lookup['daysperunit']
			else:
				cost=0

			costs.append({'step':step, 'item':item, 'budget':cost, 'description': description})

		return costs

	def getdirectcosts(self,step):
		directcosts = self.directcosts
		costs = []

		for directcost in directcosts:
			freq = directcost.get('frequency','oneoff')
			applystep = directcost.get('step',0)
			item = directcost.get('item','unspecified')
			cost = directcost.get('cost',0)
			description = directcost.get('description', '')
			type_desc = directcost.get('type', '2. Standard')

			if freq=='monthly' or (freq=='oneoff' and applystep==step) or (freq=='annual' and (step-applystep) % 12 == 0):
				cost=cost
			else:
				cost=0

			costs.append({'step':step, 'item':item, 'budget':cost, 'description': description, 'type':type_desc})

		return costs

	def getstaffcosts(self, step=None):

		# Assuming FullCostRecovery is a class you have defined elsewhere
		fcr_policy = next((policy for policy in self.policies if isinstance(policy, FullCostRecovery)), None)

		def getstep(step):
			stepregister = []
			for person in self.staff:

				# Extend register with breakdown, directly setting 'name' for each entry
				breakdown = person.getbreakdown(step)
				for entry in breakdown:
						entry["name"] = person.name
				register.extend(breakdown)

				# Assuming fcr_policy is not None and getfcr method returns a list of dicts
				if fcr_policy is not None:
						fcr_entries = fcr_policy.getfcr(person, step)
						for entry in fcr_entries:
								entry["name"] = person.name

						stepregister.extend(fcr_entries)

			return(stepregister)

		register = []
		if step is not None:
			register.extend(getstep(step))
		else:
			for step in range(self.term):
				register.extend(getstep(step))


		db = pd.DataFrame(register)
		return db

	def getbudget(self):
		budget=[]
		for i in range(self.term):
			directcosts = self.getdirectcosts(i)
			supportcosts = self.getsupports(i)
			budget.extend(directcosts)
			budget.extend(supportcosts)

			for staff in self.staff:
				budget.extend(staff.getbreakdown(i))

		for policy in self.policies:
			if hasattr(policy, 'getbudget'):
				if callable(policy.getbudget):
					budget.extend(policy.getbudget())


		df = pd.DataFrame(budget) #keep unpivoted while still working on the dataframe
		return(df)

	def getbudgetadjusted(self):
		df = self.getbudget()
		if 'step' in df.columns:
			df['step'] = df['step'] + self.startstep
		return(df)


	def getsalarycosts(self, step):
		cost = 0
		for worker in self.staff:
			cost += worker.getMonthSalaryCost(step)
			#print(f'worker {worker.name} costs {worker.getMonthSalaryCost(step)} per month')

		return (cost)

	def addstaff(self, staff):
		self.staff.append(staff)

	def sweep_policies(self, step):
		for policy in self.policies:
			policy.calculate(step)


	def start(self):
		for i in range(self.term):
			self.income_thismonth = self.costs_thismonth = 0
			self.calculate(i)
			self.sweep_policies(i)
			self.income += self.income_thismonth
			self.cost += self.costs_thismonth

			consoldacc = self.portfolio.consolidated_account

			consoldacc.update({
			 'type': 'expenditure',
			 'title': 'project costs',
			 'project': self.name,
			 'amount': self.costs_thismonth
			})
			consoldacc.update({
			 'type': 'income',
			 'title': 'project income',
			 'project': self.name,
			 'amount': self.income_thismonth
			})


			yield self.env.timeout(1)

		printtimestamp(self.env)
		print(
		 f"Project {self.name} cost {self.cost:.2f} and generated {self.income:.2f} with budget {self.budget:.2f}"
		)

Let's set up the Consolidated Account and the Portfolio that will hold all the projects.

In [ ]:
# @title Central accounts
class ConsolidatedAccount:

		def __init__(self, env):
				self.env = env
				self.total_capital = 0
				self.total_payments = 0
				self.total_income = 0
				self.balance = 0
				#self.projects = []
				self.register = []

		def update(self, transaction):
				transaction['amount'] = float(transaction['amount'])
				if transaction['type'] == 'expenditure':
						self.total_payments += transaction['amount']
				if transaction['type'] == 'income':
						self.total_income += transaction['amount']
						transaction['amount'] = -transaction['amount']

				self.balance = self.total_income - self.total_payments

				transaction['date'] = self.env.now
				transaction['balance'] = self.balance
				self.register.append(transaction)

		def report(self):
				print(
						f"Consolidated Account Report: Payments to date: {self.total_payments:.2f}, Income to date: {self.total_income:.2f}, Balance: {self.balance:.2f}"
				)


class Portfolio:

		def __init__(self, env, name='My Portfolio'):
				self.env = env
				self.name = name
				self.consolidated_account = ConsolidatedAccount(env)
				self.projects = []

		def counter(self):
				for i in range(1, 31):
						month = get_current_month(start_month='apr', month=self.env.now)
						print(f"\nMonth: {i} {month}")
						yield self.env.timeout(1)

		def set_event(self, event):
				e = self.env.event()
				e.details = event
				yield self.env.timeout(event["time"])
				printtimestamp(
						self.env)  # Assuming this is a function you have defined elsewhere

				message = event.get('message',event.get('name','new project'))

				print(f'Event {message} succeeds')
				e.succeed()
				self.env.process(self.create_project(**event))

		def set_portfolio(self, events):
				for event in events:
						self.env.process(self.set_event(event))

		def getbudget(self):
				data = {
					'item': [],
					'step': [],
					'budget': []
				}
				consol_budget = pd.DataFrame(data)

				for prj in self.projects:
					budget = prj.getbudgetadjusted()
					consol_budget = pd.concat([consol_budget, budget], ignore_index=True)

				return(consol_budget)


		def list_projects(self):
				projects = self.projects
				data = []
				for prj in projects:
						data.append({
								k: v
								for k, v in prj.__dict__.items()
								if isinstance(v, (str, int, float, bool))
						})
				df = pd.DataFrame(data)

				return df  # Return the DataFrame instead of printing it

		def run(self, until):
				#self.env.process(self.counter())
				self.env.run(until=until)

		def list_transactions(self):
				transactions = self.consolidated_account.register
				# Convert the list of dictionaries to a DataFrame
				df = pd.DataFrame(transactions)
				# Print the DataFrame as a table
				#df.head(40)
				self.consolidated_account.report()
				return(df)


		def create_project(self, cls=project, **kwargs):
				prj = cls(self, self.env, **kwargs)  # Use self.env here

				self.projects.append(prj)

				staff_names = ', '.join(person.name for person in prj.staff)
				print(
						f"Project {prj.name} created with budget {prj.budget:.2f} and assigned staff {staff_names}"
				)
				yield self.env.timeout(1)  # Use self.env here

		def finance(self, term, capital, rate=0.05):
				repayment = capital / term
				account = capital
				print(f'New capital received {capital}')
				self.consolidated_account.update({
						'type': 'income',
						'title': 'finance capitalisation',
						'project': 'headoffice',
						'amount': capital
				})
				totpay = 0
				for i in range(term):
						interest = rate * account
						account = account - repayment
						payment = repayment + interest
						totpay += payment
						self.consolidated_account.update({
								'type': 'expenditure',
								'title': 'finance servicing',
								'project': 'headoffice',
								'amount': payment
						})
						#print(f"{i+1} out of {term} paid {payment:.2f}, remaining loan: {account:.2f}")
						yield self.env.timeout(1)

				printtimestamp(self.env)
				print(f"Finance: Final account {account:.2f}, total paid {totpay:.2f}")




In [ ]:
# @title Policies
# %%writefile 'Policies.py'
class Policy():
  def __init__(self, env, prj, **kwargs):
    self.env = env
    self.prj = prj

class FullCostRecovery(Policy):
  def __init__(self, env, prj, **kwargs):
    super().__init__(env, prj, **kwargs)

    self.fcr = self.getfcrdata()
    self.register = []

  def getfcrdata(self):
    fcr=[]
    for item in FCRDATA:
      fcr.append(item)

    return fcr


  def getfcr(self, person, step):
    register=[]
    linemanagerrate = person.linemanagerrate
    fte = person.fte

    for item in self.fcr:
      itemname = item['item']
      daysperfte = item['daysperfte'] * fte
      dayrate = linemanagerrate if itemname == 'Line Management' else item['dayrate']
      frequency = item['frequency']

      cost = person.fte*daysperfte*dayrate
      if frequency == 'oneoff':
        cost = cost if step==0 else 0

      if frequency == 'monthly':
        cost = cost

      if frequency == 'annual':
        cost = cost if step % 12 == 0 else 0

      register.append({'step': step, 'item': itemname, 'budget': cost, 'type':'3. FullCostRecovery'}) # FCR HERE________________________
    return(register)

  def calcfcr(self, person, step):
    register = self.getfcr(person, step)
    self.register.extend(register)
    result = (sum(item['budget'] for item in register if 'budget' in item))
    return((result))

  def calculate(self, step):
    totalcost=0
    for person in self.prj.staff:
      totalcost += self.calcfcr(person, step)

    prj=self.prj
    prj.costs_thismonth+=totalcost

  def getbudget(self):
    return(self.register)

class Grant(Policy):
  def __init__(self, env, prj, **kwargs):
    super().__init__(env, prj, **kwargs)
    self.amount = kwargs.get('amount',0)
    self.fund = kwargs.get('fund','unspecified')
    self.startstep = kwargs.get('step', 0)
    self.register = []

  def calculate(self, step):
    prj = self.prj
    amount = self.amount
    if step == self.startstep:
      prj.income_thismonth += amount

      self.register.append({'item': f'{self.fund} grant', 'step': step, 'budget': -amount, 'type':'4. Funding'})

  def getbudget(self):
    return(self.register)


class Subsidy(Policy):

  def calculate(self, step):
    payment=100000
    prj=self.prj
    prj.income_thismonth+=payment
    prj.consolidated_account.update({
        'type': 'income',
        'title': 'government subsidy',
        'project': prj.name,
        'amount': payment
    })

class Rename(Policy):
  def calculate(self, step):
    prj=self.prj
    prj.name = f'Fancy project in step {step}'

class Finance(Policy):
  def __init__(self, env, prj, **kwargs):
    super().__init__(env, prj, **kwargs)
    #kwargs = prj.kwargs
    self.term = kwargs.get('term',prj.term)
    self.account = self.capital = kwargs.get('capital',0)
    self.rate = kwargs.get('rate',0)
    self.consolidated_account = prj.consolidated_account

    self.totpay = 0
    print(f'New capital received {self.capital}')
    self.consolidated_account.update({
		 'type': 'income',
		 'title': 'finance capitalisation',
		 'project': 'headoffice',
		 'amount': self.capital
		})

  def calculate(self, step):
    repayment = self.capital / self.term
    interest = self.rate * self.account
    self.account -= repayment
    payment = repayment + interest
    self.totpay += payment
    self.consolidated_account.update({
		    'type': 'expenditure',
		    'title': 'finance servicing',
		    'project': 'headoffice',
		    'amount': payment
		})

    if step == self.term-1:
      self.finalize()

  def finalize(self):
    printtimestamp(self.env)
    print(
    f"Finance: Final account {self.account:.2f}, total paid {self.totpay:.2f}")


class CarbonFinancing(Policy):

  def __init__(self, env, prj, **kwargs):
    #kwargs = prj.kwargs
    self.prj = prj
    self.budget = prj.budget
    self.investment = kwargs.get('investment')
    self.tree_planting_cost_per_unit = kwargs.get('tree_planting_cost_per_unit')
    self.carbon_credit_per_unit = kwargs.get('carbon_credit_per_unit')
    self.trees_planted = self.calculate_trees_planted()
    self.carbon_credits_generated = self.calculate_carbon_credits()
    self.prj.consolidated_account.update({
			 'type': 'expenditure',
			 'title': 'capital cost tree planting',
			 'project': self.prj.name,
			 'amount': self.investment-self.budget
		})
    print(f'Trees planted: {self.trees_planted:.0f} will generate {self.calculate_carbon_credits():.0f} carbon credits over 40 years worth £{self.calculate_carbon_income():.2f}')

  def calculate_trees_planted(self):
		# Assumes all investment goes to tree planting after removing the budget
    return (self.investment-self.budget) / self.tree_planting_cost_per_unit

  def calculate_carbon_credits(self):
		# Calculates carbon credits based on the number of trees planted

    unitpertreelifetime=1.1 #average tree sequesters 1.1 tonnes over 40 years lifetime.
    return self.trees_planted * unitpertreelifetime

  def calculate_carbon_income(self):
    return self.calculate_carbon_credits()*self.carbon_credit_per_unit

  def report(self):
		# Reports on the carbon financing arrangement
    return {
		 "investment": self.investment,
		 "trees_planted": self.trees_planted,
		 "carbon_credits_generated": self.carbon_credits_generated
		}

  def calculate(self, step):
    # self.costs_thismonth = self.getsalarycosts(step) + self.directcosts  #self.budget / self.term

    if step==0:
      carbonincome = self.investment #Grab the full investment in the first year only
    else:
      carbonincome = 0

    self.prj.income_thismonth += carbonincome
    return